# Predict Future Sales in Russia

- https://www.kaggle.com/c/competitive-data-science-predict-future-sales/

### Data files
- item_categories.csv
- items.csv
- sales_train.csv
- sample_submission.csv
- shops.csv
- test.csv

In [1]:
import pandas as pd

In [2]:
# definir parâmetros extras
pd.set_option('precision', 4)
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

In [45]:
input_prefix = 'data/'

translate = False # original em russo
#translate = True # traduzir para inglês

trfile = '-translated' if translate else ''
trcol = '_translated' if translate else ''

## Shops

In [46]:
df_shops = pd.read_csv(input_prefix + 'shops' + trfile + '.csv', index_col='shop_id')
print('shape:', df_shops.shape)
df_shops.head()

shape: (60, 1)


,shop_name
shop_id,
0,"!Якутск Орджоникидзе, 56 фран"
1,"!Якутск ТЦ ""Центральный"" фран"
2,"Адыгея ТЦ ""Мега"""
3,"Балашиха ТРК ""Октябрь-Киномир"""
4,"Волжский ТЦ ""Волга Молл"""


In [47]:
df_shops.describe()

,shop_name
count,60
unique,60
top,"СПб ТК ""Невский Центр"""
freq,1


In [48]:
df_shops.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 60 entries, 0 to 59
Data columns (total 1 columns):
shop_name    60 non-null object
dtypes: object(1)
memory usage: 720.0+ bytes


### Extract city name from the shop name

- 'Москва ТРК "Атриум"' => 'Москва'
- 'Н.Новгород ТРЦ "Фантастика" => 'Н.Новгород'

In [49]:
df_shops['city_name'] = df_shops['shop_name' + trcol].apply(lambda s: s.split()[0])

df_shops.head()

,shop_name,city_name
shop_id,,
0,"!Якутск Орджоникидзе, 56 фран",!Якутск
1,"!Якутск ТЦ ""Центральный"" фран",!Якутск
2,"Адыгея ТЦ ""Мега""",Адыгея
3,"Балашиха ТРК ""Октябрь-Киномир""",Балашиха
4,"Волжский ТЦ ""Волга Молл""",Волжский


In [50]:
df_shops.groupby('city_name').count().head(10)

,shop_name
city_name,
!Якутск,2
Адыгея,1
Балашиха,1
Волжский,1
Вологда,1
Воронеж,3
Выездная,1
Жуковский,2
Интернет-магазин,1


In [51]:
df_shops['city_code'] = (
    df_shops['city_name'].astype('category').cat.codes + 1
).astype('category')

df_shops.head()

,shop_name,city_name,city_code
shop_id,,,
0,"!Якутск Орджоникидзе, 56 фран",!Якутск,1
1,"!Якутск ТЦ ""Центральный"" фран",!Якутск,1
2,"Адыгея ТЦ ""Мега""",Адыгея,2
3,"Балашиха ТРК ""Октябрь-Киномир""",Балашиха,3
4,"Волжский ТЦ ""Волга Молл""",Волжский,4


In [52]:
df_shops.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 60 entries, 0 to 59
Data columns (total 3 columns):
shop_name    60 non-null object
city_name    60 non-null object
city_code    60 non-null category
dtypes: category(1), object(2)
memory usage: 2.2+ KB


In [53]:
df_shops.describe()

,shop_name,city_name,city_code
count,60,60,60
unique,60,32,32
top,"СПб ТК ""Невский Центр""",Москва,15
freq,1,13,13


## Item categories

In [54]:
df_categories = pd.read_csv(input_prefix + 'item_categories' + trfile + '.csv', index_col='item_category_id')
print('shape:', df_categories.shape)
df_categories.head()

shape: (84, 1)


,item_category_name
item_category_id,
0,PC - Гарнитуры/Наушники
1,Аксессуары - PS2
2,Аксессуары - PS3
3,Аксессуары - PS4
4,Аксессуары - PSP


In [55]:
df_categories.describe()

,item_category_name
count,84
unique,84
top,Кино - Blu-Ray 3D
freq,1


### Extract group and subgroup names from item category name

- 'Игровые консоли - PS4' => 'Игровые консоли'
- 'Карты оплаты - Windows (Цифра)' => 'Карты оплаты'
- 'Книги - Комиксы, манга' => 'Книги'

In [56]:
df_categories['group_name'] = \
    df_categories['item_category_name' + trcol].apply(
        lambda s: s.split(' - ')[0].split(' (')[0].upper())

df_categories.head()

,item_category_name,group_name
item_category_id,,
0,PC - Гарнитуры/Наушники,PC
1,Аксессуары - PS2,АКСЕССУАРЫ
2,Аксессуары - PS3,АКСЕССУАРЫ
3,Аксессуары - PS4,АКСЕССУАРЫ
4,Аксессуары - PSP,АКСЕССУАРЫ


In [57]:
df_categories.groupby('group_name').count().head(10)

,item_category_name
group_name,
PC,1
АКСЕССУАРЫ,7
БИЛЕТЫ,1
ДОСТАВКА ТОВАРА,1
ИГРОВЫЕ КОНСОЛИ,8
ИГРЫ,8
ИГРЫ ANDROID,1
ИГРЫ MAC,1
ИГРЫ PC,4


In [58]:
def extract_subgroup(s):
    gs = s.split(' - ')
    if len(gs) > 1:
        gs2 = gs[1].split(' (')
        return gs2[0].upper()
    else:
        return ''
    #return gs[1] if len(gs) > 1 else ''
    
df_categories['subgroup_name'] = \
    df_categories['item_category_name' + trcol].apply(
        lambda s: extract_subgroup(s))

df_categories.head()

,item_category_name,group_name,subgroup_name
item_category_id,,,
0,PC - Гарнитуры/Наушники,PC,ГАРНИТУРЫ/НАУШНИКИ
1,Аксессуары - PS2,АКСЕССУАРЫ,PS2
2,Аксессуары - PS3,АКСЕССУАРЫ,PS3
3,Аксессуары - PS4,АКСЕССУАРЫ,PS4
4,Аксессуары - PSP,АКСЕССУАРЫ,PSP


In [59]:
df_categories.groupby('subgroup_name').count().head(10)

,item_category_name,group_name
subgroup_name,,
,7,7
1С:ПРЕДПРИЯТИЕ 8,1,1
BLU-RAY,1,1
BLU-RAY 3D,1,1
BLU-RAY 4K,1,1
CD ЛОКАЛЬНОГО ПРОИЗВОДСТВА,1,1
CD ФИРМЕННОГО ПРОИЗВОДСТВА,1,1
DVD,1,1
LIVE!,2,2


In [60]:
df_categories['group_code'] = (
    df_categories['group_name'].astype('category').cat.codes + 1).astype('category')

df_categories['subgroup_code'] = (
    df_categories['subgroup_name'].astype('category').cat.codes + 1).astype('category')

df_categories.head()

,item_category_name,group_name,subgroup_name,group_code,subgroup_code
item_category_id,,,,,
0,PC - Гарнитуры/Наушники,PC,ГАРНИТУРЫ/НАУШНИКИ,1,30
1,Аксессуары - PS2,АКСЕССУАРЫ,PS2,2,12
2,Аксессуары - PS3,АКСЕССУАРЫ,PS3,2,13
3,Аксессуары - PS4,АКСЕССУАРЫ,PS4,2,14
4,Аксессуары - PSP,АКСЕССУАРЫ,PSP,2,16


In [61]:
df_categories.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 84 entries, 0 to 83
Data columns (total 5 columns):
item_category_name    84 non-null object
group_name            84 non-null object
subgroup_name         84 non-null object
group_code            84 non-null category
subgroup_code         84 non-null category
dtypes: category(2), object(3)
memory usage: 4.9+ KB


In [62]:
df_categories.describe()

,item_category_name,group_name,subgroup_name,group_code,subgroup_code
count,84,84,84,84,84
unique,84,18,55,18,55
top,Кино - Blu-Ray 3D,КНИГИ,,12,1
freq,1,13,7,13,7


## Items

In [63]:
df_items = pd.read_csv(input_prefix + 'items' + trfile + '.csv', index_col='item_id')
print('shape:', df_items.shape)
df_items.head()

shape: (22170, 2)


,item_name,item_category_id
item_id,,
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,40
1,!ABBYY FineReader 12 Professional Edition Full...,76
2,***В ЛУЧАХ СЛАВЫ (UNV) D,40
3,***ГОЛУБАЯ ВОЛНА (Univ) D,40
4,***КОРОБКА (СТЕКЛО) D,40


In [67]:
if not translate:
    df_items['item_category_id'] = df_items['item_category_id'].astype('category')

In [68]:
df_items.describe()

,item_name,item_category_id
count,22170,22170
unique,22170,84
top,Win Pro 8 32-bit/64-bit Russian VUP Russia Onl...,40
freq,1,5035


In [69]:
import re

def extract_main_subject(str):
    s = str.upper()
    # remover caracteres do começo => !"*/
    s = re.sub("^[!*/\"]+ ?", "", s)
    #FIXME: remover termo "THE" do nome
    s = re.sub("^THE ", "", s) # translated
    # obter primeira palavra em maiúsculo
    s = s.split()[0]
    # substituir caracteres => '`’
    s = re.sub("['`’]", "_", s)
    # remover caracteres do fim da palavra => :.®,!
    s = re.sub("[:.,!®]$", "", s)
    return s
    
df_items['subject_name'] = \
    df_items['item_name' + trcol].apply(
        lambda s: extract_main_subject(s))

df_items.head()

,item_name,item_category_id,subject_name
item_id,,,
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,40,ВО
1,!ABBYY FineReader 12 Professional Edition Full...,76,ABBYY
2,***В ЛУЧАХ СЛАВЫ (UNV) D,40,В
3,***ГОЛУБАЯ ВОЛНА (Univ) D,40,ГОЛУБАЯ
4,***КОРОБКА (СТЕКЛО) D,40,КОРОБКА


In [70]:
df_items.tail()

,item_name,item_category_id,subject_name
item_id,,,
22165,"Ядерный титбит 2 [PC, Цифровая версия]",31,ЯДЕРНЫЙ
22166,Язык запросов 1С:Предприятия [Цифровая версия],54,ЯЗЫК
22167,Язык запросов 1С:Предприятия 8 (+CD). Хрустале...,49,ЯЗЫК
22168,Яйцо для Little Inu,62,ЯЙЦО
22169,Яйцо дракона (Игра престолов),69,ЯЙЦО


In [71]:
df_items.groupby('subject_name').count().head(10)

,item_name,item_category_id
subject_name,,
007,5,5
1+1,2,2
10,6,6
100,20,20
1000,2,2
10000,1,1
101,8,8
11,2,2
11-11-11,1,1


In [72]:
df_items['subject_code'] = (
    df_items['subject_name'].astype('category').cat.codes + 1).astype('category')

df_items.head()

,item_name,item_category_id,subject_name,subject_code
item_id,,,,
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,40,ВО,1804
1,!ABBYY FineReader 12 Professional Edition Full...,76,ABBYY,112
2,***В ЛУЧАХ СЛАВЫ (UNV) D,40,В,1615
3,***ГОЛУБАЯ ВОЛНА (Univ) D,40,ГОЛУБАЯ,1959
4,***КОРОБКА (СТЕКЛО) D,40,КОРОБКА,2631


In [73]:
df_items.tail()

,item_name,item_category_id,subject_name,subject_code
item_id,,,,
22165,"Ядерный титбит 2 [PC, Цифровая версия]",31,ЯДЕРНЫЙ,4647
22166,Язык запросов 1С:Предприятия [Цифровая версия],54,ЯЗЫК,4649
22167,Язык запросов 1С:Предприятия 8 (+CD). Хрустале...,49,ЯЗЫК,4649
22168,Яйцо для Little Inu,62,ЯЙЦО,4650
22169,Яйцо дракона (Игра престолов),69,ЯЙЦО,4650


In [74]:
df_items.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22170 entries, 0 to 22169
Data columns (total 4 columns):
item_name           22170 non-null object
item_category_id    22170 non-null category
subject_name        22170 non-null object
subject_code        22170 non-null category
dtypes: category(2), object(2)
memory usage: 578.4+ KB


In [75]:
df_items.describe()

,item_name,item_category_id,subject_name,subject_code
count,22170,22170,22170,22170
unique,22170,84,4656,4656
top,Win Pro 8 32-bit/64-bit Russian VUP Russia Onl...,40,ФИГУРКА,4349
freq,1,5035,599,599


In [ ]:
#TODO: join items + categories:item_category_id => group_code, subgroup_code

## Sales (training)

In [76]:
dateparse = lambda x: pd.datetime.strptime(x, '%d.%m.%Y')
df_train = pd.read_csv(input_prefix + 'sales_train-1k.csv', parse_dates=['date'], date_parser=dateparse)
print('shape:', df_items.shape)
df_train.head()

shape: (22170, 4)


,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,2013-01-02,0,59,22154,999.00,1.0
1,2013-01-03,0,25,2552,899.00,1.0
2,2013-01-05,0,25,2552,899.00,-1.0
3,2013-01-06,0,25,2554,1709.05,1.0
4,2013-01-15,0,25,2555,1099.00,1.0


In [82]:
for col in ('shop_id', 'item_id'):
    df_train[col] = df_train[col].astype('category')

In [83]:
df_train.describe()

,date_block_num,item_price,item_cnt_day
count,1000.0,1000.0000,1000.0000
mean,0.0,908.4799,1.1900
std,0.0,822.7630,0.7671
min,0.0,58.0000,-1.0000
25%,0.0,299.0000,1.0000
50%,0.0,599.0000,1.0000
75%,0.0,1290.0000,1.0000
max,0.0,5490.0000,13.0000


In [87]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
date              1000 non-null datetime64[ns]
date_block_num    1000 non-null int64
shop_id           1000 non-null category
item_id           1000 non-null category
item_price        1000 non-null float64
item_cnt_day      1000 non-null float64
dtypes: category(2), datetime64[ns](1), float64(2), int64(1)
memory usage: 45.3 KB


In [ ]:
#TODO: converter 'date' para tipo data - OK

#TODO: mesclar com tabelas de lojas, itens e categorias

#TODO: verificar o que significam contagens negativas - OK
# It means the item is returned. You should predict that too.
# Or in the other words: it is not missing data or mistake.

In [ ]:
#TODO: criar colunas contendo dia da semana a partir de 'date'

#TODO: baixar calendário de feriados na Rússia:
#      https://www.google.com/search?q=holidays+calendar+in+russia+format%3Acsv

#TODO: criar colunas indicando feriado

#TODO: criar colunas indicando véspera ou pós-feriado

In [ ]:
#TODO: join sales + shops:shop_id => city_code
#TODO: join sales + items:item_id => subject_code, (group_code, subgroup_code)

## Sales (testing)

In [88]:
df_test = pd.read_csv(input_prefix + 'test.csv')
print('shape:', df_items.shape)
df_test.head()

shape: (22170, 4)


,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268


In [89]:
#TODO: verificar o que está sendo esperado (acumulado mensal por loja e produto?)
# item_cnt_day: number of products sold. You are predicting a monthly amount of this measure

In [91]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 214200 entries, 0 to 214199
Data columns (total 3 columns):
ID         214200 non-null int64
shop_id    214200 non-null int64
item_id    214200 non-null int64
dtypes: int64(3)
memory usage: 4.9 MB


In [90]:
df_test.describe()

,ID,shop_id,item_id
count,214200.0000,214200.0000,214200.0000
mean,107099.5000,31.6429,11019.3986
std,61834.3582,17.5619,6252.6446
min,0.0000,2.0000,30.0000
25%,53549.7500,16.0000,5381.5000
50%,107099.5000,34.5000,11203.0000
75%,160649.2500,47.0000,16071.5000
max,214199.0000,59.0000,22167.0000
